So, first we will start by importing the libraries that are required in this automation

Next , we provide the Base Url template for the case from which we will be handling API calls for all 3 versions

On manual exploration, I found that the max result that we get from one query is 10 and there are further more names in the nested location so let's state the maximum result here first

To make sure that we dont get the error of too many requests i introduce a delay.

Next, we will create a function that makes the API call with  a given prefix and test for case'a'

This function has the Parameters: 
- prefix (str): The current prefix to query.
- all_names (set): A set to store all unique names.
- request_counter (list): counter to track API requests.
- base_url

The function starts by taking a delay to avoid too many requests on the server, then it sends a query using the current prefix and increases the counter for every API call made. If the results come back with fewer entries than the maximum allowed, that means there aren’t any hidden names in that branch and saves those names. But if the results hit the maximum limit, it means there might be more names left so the function digs deeper by adding each letter of the alphabet to the prefix. It keeps track of every suggestion it finds along the way.

Now, we make the main function that will be used to solve the whole statement 

on putting the whole code together 

In [ ]:
import requests
import time
import string
import threading

BASE_URL_TEMPLATE = "http://35.200.185.69:8000/{}/autocomplete?query="
MAX_RESULTS = 10  
RATE_LIMIT_DELAY = 0.3  

def get_suggestions(prefix, base_url):
    url = f"{base_url}{prefix}"
    
    try:
        print(f"Fetching {url} ...", flush=True)  # Debugging print
        response = requests.get(url)
        if response.status_code == 200:
            return response.json().get("results", [])
        elif response.status_code == 429:
            print(f"Rate limited for prefix '{prefix}'. Waiting...", flush=True)
            time.sleep(1)
            return get_suggestions(prefix, base_url)
        else:
            print(f"Unexpected status {response.status_code} for prefix '{prefix}'", flush=True)
            return []
    except Exception as e:
        print(f"Error for prefix '{prefix}': {e}", flush=True)
        return []

def recursive_search(prefix, all_names, request_counter, base_url):
    time.sleep(RATE_LIMIT_DELAY)
    
    suggestions = get_suggestions(prefix, base_url)
    request_counter[0] += 1

    print(f"Prefix '{prefix}' returned {len(suggestions)} results.", flush=True)

    if not suggestions:
        return
    
    if len(suggestions) < MAX_RESULTS:
        all_names.update(suggestions)
    else:
        for char in string.ascii_lowercase:
            recursive_search(prefix + char, all_names, request_counter, base_url)
    
    all_names.update(suggestions)

def fetch_data_for_version(version):
    base_url = BASE_URL_TEMPLATE.format(version)
    all_names = set()
    request_counter = [0]

    print(f"Starting extraction for API version: {version}", flush=True)
    
    for char in string.ascii_lowercase:
        recursive_search(char, all_names, request_counter, base_url)
    
    print(f"\n--- Extraction Complete for {version} ---", flush=True)
    print(f"Total API Requests Made: {request_counter[0]}", flush=True)
    print(f"Total Unique Names Found: {len(all_names)}", flush=True)

    with open(f"names_{version}.txt", "w") as file:
        for name in sorted(all_names):
            file.write(name + "\n")

def main():
    api_versions = ['v1', 'v2', 'v3']
    threads = []

    for version in api_versions:
        print(f"Starting thread for {version}...", flush=True)
        thread = threading.Thread(target=fetch_data_for_version, args=(version,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
        print(f"Thread for version {version} finished.", flush=True)

    print("\n--- All API Versions Processed Successfully ---", flush=True)

main()